<a href="https://colab.research.google.com/github/manhhung14/echocardiograms/blob/main/Project_Machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Khởi tạo thư viện**

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import cv2
import numpy as np
#from matplotlib import pyplot as plt
from google.colab import drive
import os
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from collections import namedtuple
from sklearn.metrics import classification_report



**Lấy dữ liệu**

In [2]:
!git clone https://github.com/manhhung14/echocardiograms
traindir = "/content/echocardiograms/DATA_CHAMBER_2021/train"
testdir = "/content/echocardiograms/DATA_CHAMBER_2021/test"

Cloning into 'echocardiograms'...
remote: Enumerating objects: 8337, done.
remote: Counting objects: 100% (8337/8337), done.
remote: Compressing objects: 100% (8334/8334), done.
remote: Total 8337 (delta 2), reused 8328 (delta 2), pack-reused 0
Receiving objects: 100% (8337/8337), 488.02 MiB | 34.16 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Checking out files: 100% (8327/8327), done.


**Chuẩn bị dữ liệu**

In [3]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])
def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

#TrainTest = namedtuple('TrainTest', ['train', 'test'])

def prepare_data():
  
  transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    #transforms.RandomCrop(224, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
  ])
  transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
  ])
  trainset = torchvision.datasets.CIFAR10(root=traindir, download=True, train=True, transform=transform_train)
  testset = torchvision.datasets.CIFAR10(root=testdir, download=True, train=False, transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

**Train & Test**

In [4]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 60
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

**Thực thi**

In [19]:
def main(PATH='./model.pth', model_in=None):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  if model_in == 'vgg16':  
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=3, bias=True)
  elif model_in == 'resnet18':
    model = torchvision.models.resnet18()
    model.fc = torch.nn.modules.linear.Linear(in_features=512, out_features=3, bias=True) 
  else: 
    
    pass

  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))
    return model

In [20]:
m = main(PATH="./vgg16.pth", model_in='vgg16')

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


IndexError: ignored